# Web Scrapping
## S&P 100 Wiki and 10-K SEC EDGARS forms of Apple and Intel


**Suraj Kumar** 

**Current Job** \
*Big Data Model Developer @ American Express*

**Post Graduation**  
*MA Economics(Final Year)*  
*Department of Economics*  
**Delhi School of Economics**  
*India*

**Under graduation**    
*B.Tech in Civil Engineering from __Indian Institute of Technology, Delhi (IIT Delhi)__*


## Introduction

This notebook contains the python code to complete a coding assignment given by *Prof. John Gallemore*. The assignment mainly testes out the coding skill of the candidate with regard to web scrapping.

**Task 1: Description**\
In this python package *requests* has been used to retrieve a table of S&P 100 companies from their wikipedia page. After retreiving the table, the company name columns is fuzzy matched with set of company names from from local gvkeys.csv file by using python *fuzzywuzzy* package.

**Task 2:Description**\
In this task, 10-K Forms of Intel and Apple for the fiscal years of 2014 to 2018 have been retrieved from SEC EDGAR website. The the next step, Risk Section portion of the 10-K form is analyzed to get the following information

1. The number of words in *Risk Section*
2. The number of times the word "competition" repeats in the *Risk Section*.
3. Exports the text of *Risk Section* to txt file
4. Python package *nltk* is used to remove all the stopwords
5. Notes occurence of the word *patents* and its variation such as patenting, patents, etc. It notes each repetition as word preceding, occurence found and word following.



##  Task 1: Retriving Data from S&P 100 wikipedia page

In [522]:
# importing libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
#import re # importing regex
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
 


# Setting up options on display of row and columns in Pandas data frame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',-1)

## To output multiple objects in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\suraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\suraj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
# retriving the data from the website

## defining the url
url = "https://en.wikipedia.org/wiki/S%26P_100"

## retriving the url
r = requests.get(url)

In [6]:
## Checking the status of the request
r.status_code

# 200 means success

200

In [9]:
# converting to text
html_doc = r.text

In [ ]:
# printing the html doc
#print(html_doc)

# converting to Beautiful Soup format

html_doc_soup = BeautifulSoup(html_doc)

In [2]:
#html_doc_soup

In [14]:
# Extracting the table of companies
table = html_doc_soup.find('table',{'class':'wikitable sortable'})
#table = soup.find('table', {'class': 'details'})

In [1]:
#table

In [138]:
# Defining the firms table

pd_firms = pd.DataFrame(columns = ['Symbol','Name','Link'])

In [139]:
pd_firms

,Symbol,Name,Link


In [140]:
#pd_firms.at[0,'Symbol'] = 'a'
pd_firms

,Symbol,Name,Link


In [146]:
row_marker = 0
for row in table.find_all('tr'):
    columns = row.find_all('td')
    # Filter so as to not run the following commands on the first element of the 
    # row as it contains only 'th' tags so find_all('td') will be null.
    if(len(columns)!= 0): 
        # Getting the Symbol
        symbol_cell = columns[0]
        pd_firms.at[row_marker,'Symbol'] = symbol_cell.get_text().rstrip()
        
        # Extracing the tag from the the second element in column list
        name_and_link_tag_cell = columns[1].find_all('a')[0]
        
        # Getting the Name
        pd_firms.at[row_marker,'Name'] = name_and_link_tag_cell.get('title')

        # Getting the link
        pd_firms.at[row_marker,'Link'] = "https://en.wikipedia.org"+name_and_link_tag_cell.get('href')
        row_marker = row_marker + 1
    
    #print(row_marker)
pd_firms

,Symbol,Name,Link
0,AAPL,Apple Inc.,https://en.wikipedia.org/wiki/Apple_Inc.
1,ABBV,AbbVie Inc.,https://en.wikipedia.org/wiki/AbbVie_Inc.
2,ABT,Abbott Laboratories,https://en.wikipedia.org/wiki/Abbott_Laboratories
3,ACN,Accenture,https://en.wikipedia.org/wiki/Accenture
4,ADBE,Adobe Inc.,https://en.wikipedia.org/wiki/Adobe_Inc.
5,AGN,Allergan,https://en.wikipedia.org/wiki/Allergan
6,AIG,American International Group,https://en.wikipedia.org/wiki/American_International_Group
7,ALL,Allstate,https://en.wikipedia.org/wiki/Allstate
8,AMGN,Amgen,https://en.wikipedia.org/wiki/Amgen
9,AMZN,Amazon.com,https://en.wikipedia.org/wiki/Amazon.com


In [155]:
## importing the gvkeys.csv now
gvkeys_df = pd.read_csv("names_gvkeys.csv")

# Checking number of columns and rows
gvkeys_df.shape

(112086, 2)

In [226]:
# Checking the head
gvkeys_df.head(11)

# Removing the duplicates in gvkeys dataframe
gvkeys_nodup_df = gvkeys_df.drop_duplicates()

# Checking the shape of this dataframe
gvkeys_nodup_df.shape

# Checking the head
gvkeys_nodup_df.head(10)

,gvkey,conm
0,1004,AAR CORP
1,1004,AAR CORP
2,1004,AAR CORP
3,1004,AAR CORP
4,1004,AAR CORP
5,1004,AAR CORP
6,1004,AAR CORP
7,1004,AAR CORP
8,1004,AAR CORP
9,1013,ADC TELECOMMUNICATIONS INC


(16876, 2)

,gvkey,conm
0,1004,AAR CORP
9,1013,ADC TELECOMMUNICATIONS INC
10,1045,AMERICAN AIRLINES GROUP INC
19,1050,CECO ENVIRONMENTAL CORP
28,1062,ASA GOLD AND PRECIOUS METALS
46,1072,AVX CORP
56,1075,PINNACLE WEST CAPITAL CORP
65,1076,AARON'S INC
74,1078,ABBOTT LABORATORIES
83,1082,SERVIDYNE INC


In [227]:
# Checking for Unique Counts
gvkeys_nodup_df.nunique()


# Reseting the index of gvkeys_nodup_df
gvkeys_nodup_df = gvkeys_nodup_df.reset_index()
gvkeys_nodup_df.head()

gvkey    16876
conm     16876
dtype: int64

,index,gvkey,conm
0,0,1004,AAR CORP
1,9,1013,ADC TELECOMMUNICATIONS INC
2,10,1045,AMERICAN AIRLINES GROUP INC
3,19,1050,CECO ENVIRONMENTAL CORP
4,28,1062,ASA GOLD AND PRECIOUS METALS


In [228]:
# Adding addtional columns to pd_firms dataframe

pd_firms['matched_company_name'] = ''
pd_firms['score'] = np.nan

# Checking the head
pd_firms.head()

,Symbol,Name,Link,matched_company_name,score
0,AAPL,Apple Inc.,https://en.wikipedia.org/wiki/Apple_Inc.,,NaN
1,ABBV,AbbVie Inc.,https://en.wikipedia.org/wiki/AbbVie_Inc.,,NaN
2,ABT,Abbott Laboratories,https://en.wikipedia.org/wiki/Abbott_Laboratories,,NaN
3,ACN,Accenture,https://en.wikipedia.org/wiki/Accenture,,NaN
4,ADBE,Adobe Inc.,https://en.wikipedia.org/wiki/Adobe_Inc.,,NaN


In [218]:
number_of_firms_in_SP100 = pd_firms.shape[0]
number_of_firms_in_SP100
#Strange its 101 when it should be 100

101

In [240]:
# Consedring the company names from gvkeys dataframe

strOptions = gvkeys_nodup_df['conm']

In [245]:
# This loop could have been paralel parallelized given more time
# Running a For Loop to Assign values from gvkeys_nodup_df into pd_firms dataframe
for i in range(number_of_firms_in_SP100):
    str2Match = pd_firms.iloc[i,1]
    
    # Getting the best match
    highest = process.extractOne(str2Match,strOptions)
    highest_index = highest[2]
    
    # Extracting the matched company name and score using the index from highest tuple
    matched_name = gvkeys_nodup_df['conm'].iloc[highest_index]
    score = gvkeys_nodup_df['gvkey'].iloc[highest_index]
    
    # Setting the values in pd_firms dataframe
    pd_firms.at[i,'matched_company_name'] = matched_name
    pd_firms.at[i,'score'] = score
    #print(str2Match)

In [247]:
# Looking at the dataframe
pd_firms

,Symbol,Name,Link,matched_company_name,score
0,AAPL,Apple Inc.,https://en.wikipedia.org/wiki/Apple_Inc.,APPLE INC,1690.0
1,ABBV,AbbVie Inc.,https://en.wikipedia.org/wiki/AbbVie_Inc.,ABBVIE INC,16101.0
2,ABT,Abbott Laboratories,https://en.wikipedia.org/wiki/Abbott_Laboratories,ABBOTT LABORATORIES,1078.0
3,ACN,Accenture,https://en.wikipedia.org/wiki/Accenture,ACCENTURE PLC,143357.0
4,ADBE,Adobe Inc.,https://en.wikipedia.org/wiki/Adobe_Inc.,ADOBE INC,12540.0
5,AGN,Allergan,https://en.wikipedia.org/wiki/Allergan,ALLERGAN INC,15708.0
6,AIG,American International Group,https://en.wikipedia.org/wiki/American_International_Group,AMERICAN INTERNATIONAL GROUP,1487.0
7,ALL,Allstate,https://en.wikipedia.org/wiki/Allstate,ALLSTATE CORP,28349.0
8,AMGN,Amgen,https://en.wikipedia.org/wiki/Amgen,AMGEN INC,1602.0
9,AMZN,Amazon.com,https://en.wikipedia.org/wiki/Amazon.com,AMAZON.COM INC,64768.0


In [248]:
# Exporting to csv
pd_firms.to_csv('task1_solution_Suraj_Kumar.csv',index = False)

## Task 2: Apple Inc and Intel 10-K Form study

 Assumping that Fiscal year means the year in which the fiscal ended. 
 
 * For Apple, fiscal year 2014 duration is from Oct 2013 to Sep 2014 and 
 * for Intel, fiscal Year 2014 is from Jan 2014 to Dec 2014

In [249]:
# Listing the files_url needed to process the records
apple_2014 = "https://www.sec.gov/Archives/edgar/data/320193/000119312514383437/d783162d10k.htm"
apple_2015 = "https://www.sec.gov/Archives/edgar/data/320193/000119312515356351/d17062d10k.htm"
apple_2016 = "https://www.sec.gov/Archives/edgar/data/320193/000162828016020309/a201610-k9242016.htm"
apple_2017 = "https://www.sec.gov/Archives/edgar/data/320193/000032019317000070/a10-k20179302017.htm"
apple_2018 = "https://www.sec.gov/Archives/edgar/data/320193/000032019318000145/a10-k20189292018.htm"


# Intel
intel_2014 = "https://www.sec.gov/Archives/edgar/data/50863/000005086315000015/a10kdocument12272014.htm"
intel_2015 = "https://www.sec.gov/Archives/edgar/data/50863/000005086316000105/a10kdocument12262015q4.htm"
intel_2016 = "https://www.sec.gov/Archives/edgar/data/50863/000005086317000012/a10kdocument12312016q4.htm"
intel_2017 = "https://www.sec.gov/Archives/edgar/data/50863/000005086318000007/a12302017q4-10kdocument.htm"
intel_2018 = "https://www.sec.gov/Archives/edgar/data/50863/000005086319000007/a12292018q4-10kdocument.htm"

In [250]:
apple_10k = [apple_2014, apple_2015, apple_2016, apple_2017, apple_2018]
intel_10k = [intel_2014, intel_2015, intel_2016, intel_2017, intel_2018]

In [268]:
# Starting empty dic to store the parameters of the request retrival package
ten_k_dict = {}
# Retrving the data using request package and using Beautiful soup on it
for ten_k in (apple_10k + intel_10k):
    print(ten_k)
    r = requests.get(ten_k)
    
    #Storing the results
    ten_k_dict[ten_k] = {'status_code':r.status_code,'soup': BeautifulSoup(r.text)}

https://www.sec.gov/Archives/edgar/data/320193/000119312514383437/d783162d10k.htm
https://www.sec.gov/Archives/edgar/data/320193/000119312515356351/d17062d10k.htm
https://www.sec.gov/Archives/edgar/data/320193/000162828016020309/a201610-k9242016.htm
https://www.sec.gov/Archives/edgar/data/320193/000032019317000070/a10-k20179302017.htm
https://www.sec.gov/Archives/edgar/data/320193/000032019318000145/a10-k20189292018.htm
https://www.sec.gov/Archives/edgar/data/50863/000005086315000015/a10kdocument12272014.htm
https://www.sec.gov/Archives/edgar/data/50863/000005086316000105/a10kdocument12262015q4.htm
https://www.sec.gov/Archives/edgar/data/50863/000005086317000012/a10kdocument12312016q4.htm
https://www.sec.gov/Archives/edgar/data/50863/000005086318000007/a12302017q4-10kdocument.htm
https://www.sec.gov/Archives/edgar/data/50863/000005086319000007/a12292018q4-10kdocument.htm


In [273]:
# Checking the status code of all scraping
for ten_k in (apple_10k + intel_10k):
    ten_k_dict[ten_k]['status_code']

200

200

200

200

200

200

200

200

200

200

### Task 2- Part 1

#### Task 2 Part 1 : Apple Analysis

In [569]:
# Function to do task 2 part 1 
def risk_factor_section_stats_generator(company, company_url_list, 
                                        risk_factor_section_start_string,
                                        risk_factor_section_stop_string,
                                        year_list = ['2014','2015','2016','2017','2018']):
    """
    company is company name 
    company_url_list is company url list as defined earliar
    risk_factor_section_start_string is string pattern to identify the beginning of risk section
    risk_factor_section_stop_string is to identify the end of risk section
    year list is names of year of url
    
    """
    # Creating the Dataframe to input results
    risk_factor_section_stats = pd.DataFrame(columns=['company',"year",
                                                      "risk_section_word_length",
                                                      "competition_count"])

    # Looking at one soup
    for i in range(len(year_list)):

        #Printing the year
        #print('The year is {}'.format(year_list[i]))

        # Extracing the soup
        soup = ten_k_dict[company_url_list[i]]['soup']

        # Extracting the p tags
        soup_tags = soup.find_all('p')

        if(len(soup_tags) == 0): # This conditional statment is put here to account for 
            #change in html tag structure starting from year 2016 from p to div
            soup_tags = soup.find_all('div')

        # Extracting paragraphs from these p tags
        soup_paragraphs = [par.get_text().rstrip() for par in soup_tags]

        # Extracting the starting and ending paragraph number of risk section
        for j in range(len(soup_paragraphs)):
            if(  soup_paragraphs[j].find(risk_factor_section_start_string) != -1 ) :
                start_paragraph_number = j
                #print('Starting Paragraph No is {0}'.format(start_paragraph_number))

            if( soup_paragraphs[j].find(risk_factor_section_stop_string) != -1):
                end_paragraph_number = j
                #print('Ending Paragraph No is {0}'.format(end_paragraph_number))

        # Intitiating variable to store risk section content.
        risk_section = ''
        for paragraph in soup_paragraphs[start_paragraph_number:end_paragraph_number]:
            #print(paragraph)
            risk_section = risk_section + ' ' + paragraph

        # Counting the number of words in risk section
        risk_section_words = risk_section.split()
        risk_section_number_of_words = len(risk_section_words)
        #print(risk_section_number_of_words)

        # Counting the number of times the competition repeats in risk section
        risk_section_no_period = risk_section.replace("."," ")  # minor modification to replace period
        competition_count = risk_section_no_period.count('competition') # assuming that we are only searching for "competition" and not "Competition"
        #print(competition_count)

        # Writing the risk section to txt file
        file_name = company + "_" + year_list[i] + ".txt"
        #print(file_name)
        file1 = open(file_name,"w",encoding='utf-8')
        file1.write(risk_section)
        file1.close()

        # Setting the values to a dataframe
        risk_factor_section_stats.at[i,"company"] = company
        risk_factor_section_stats.at[i,"year"] = year_list[i]
        risk_factor_section_stats.at[i,"risk_section_word_length"] = risk_section_number_of_words
        risk_factor_section_stats.at[i,"competition_count"] = competition_count

    return(risk_factor_section_stats)

In [570]:
risk_factor_section_stats_apple = risk_factor_section_stats_generator("apple",
                                                                      apple_10k,
                                                                      risk_factor_section_start_string = "The following discussion of risk factors contains forward-looking statements" ,
                                                                      risk_factor_section_stop_string = "headquarters are located in Cupertino, California")
risk_factor_section_stats_apple

,company,year,risk_section_word_length,competition_count
0,apple,2014,7710,8
1,apple,2015,7765,8
2,apple,2016,7680,8
3,apple,2017,7779,8
4,apple,2018,8082,9


In [503]:
# doing the similar thing for intel
risk_factor_section_stats_intel_2014_to_2016 = risk_factor_section_stats_generator("intel",
                                                                      intel_10k,
                                                                      risk_factor_section_start_string = "The following risks could materially and adversely" ,
                                                                      risk_factor_section_stop_string = "UNRESOLVED STAFF COMMENTS",
                                                                      year_list= ['2014','2015','2016'])
risk_factor_section_stats_intel_2014_to_2016

,company,year,risk_section_word_length,competition_count
0,intel,2014,5824,5
1,intel,2015,5833,5
2,intel,2016,6406,5


In [505]:
# Due to change in Intel 10k Form HTML strucure so doing it differently for year 2017 and 2018
intel_10k_2017_2018 = [intel_2017,intel_2018]

risk_factor_section_stats_intel_2017_2018 = risk_factor_section_stats_generator("intel",
                                                                      intel_10k_2017_2018,
                                                                      risk_factor_section_start_string = "The following risks could materially and adversely" ,
                                                                      risk_factor_section_stop_string = "In addition to disclosing financial results in accordance with GAAP",
                                                                      year_list= ['2017','2018'])
risk_factor_section_stats_intel_2017_2018

,company,year,risk_section_word_length,competition_count
0,intel,2017,6669,5
1,intel,2018,8458,6


In [506]:
## Now combining all the datasets in one data
risk_factor_section_stats = pd.concat([risk_factor_section_stats_apple,
                                         risk_factor_section_stats_intel_2014_to_2016,
                                        risk_factor_section_stats_intel_2017_2018])



In [511]:
# Printing and writing to csv the results
risk_factor_section_stats
risk_factor_section_stats.to_csv("task2_part1_risk_factor_section_stats.csv",index=False)

,company,year,risk_section_word_length,competition_count
0,apple,2014,7710,8
1,apple,2015,7765,8
2,apple,2016,7680,8
3,apple,2017,7779,8
4,apple,2018,8082,9
0,intel,2014,5824,5
1,intel,2015,5833,5
2,intel,2016,6406,5
0,intel,2017,6669,5
1,intel,2018,8458,6


### Task 2 Part 2 NLTK Analsys

In [564]:
# Defining the stop word
stop_words = set(stopwords.words('english')) 

# Setting up search word to find in filtered_sentence
search_word = 'patent'

# Settup the dataframe
patent_match_df = pd.DataFrame(columns=['file_name','word_preceding','occurence_found','word_following'])
i = 0 # setting the first row of the dataframe 

for company in ['apple','intel']:
    for year in ['2014','2015','2016','2017','2018']:
        
        ## importing the txt file
        file_name = company + "_" + year + ".txt"
        print(file_name)
        #file_name = "apple_2014.txt"
        
        file_object = open(file_name,"r",encoding='utf-8')
        data = file_object.read().replace('\n', ' ')
        data_no_period = data.replace("."," ")
        
        # Tokenings
        word_tokens = word_tokenize(data_no_period) 

        # Checking the length of word_tokens
        len(word_tokens)

        # Removing the stop words
        filtered_sentence = [w for w in word_tokens if not w in stop_words] 

        # Checking the length after remving stop words
        len(filtered_sentence)

        #chekcing
        #word_tokens[0:20]

        #checking
        #filtered_sentence[0:20]
        for j in range(len(filtered_sentence)):
            if (filtered_sentence[j].lower().find(search_word) != -1): # condtion to check the match
                patent_match_df.at[i,'file_name'] = file_name
                patent_match_df.at[i,'word_preceding'] = filtered_sentence[j-1]
                patent_match_df.at[i,'occurence_found'] = filtered_sentence[j]
                patent_match_df.at[i,'word_following'] = filtered_sentence[j+1]
                i = i + 1

apple_2014.txt


8239

5493

apple_2015.txt


8301

...

7332

4928

intel_2018.txt


9315

6231

In [565]:
patent_match_df.shape

(98, 4)

In [567]:
patent_match_df
patent_match_df.to_csv("task2_part2_patent_match.csv",index = False)

,file_name,word_preceding,occurence_found,word_following
0,apple_2014.txt,number,patents,copyrights
1,apple_2014.txt,numerous,patents,","
2,apple_2014.txt,allegations,patent,infringement
3,apple_2014.txt,",",patent,holding
4,apple_2014.txt,monetize,patents,purchased
5,apple_2014.txt,significant,patent,portfolios
6,apple_2014.txt,number,patent,claims
7,apple_2014.txt,validity,patents,intellectual
8,apple_2014.txt,one,patents,intellectual
9,apple_2015.txt,number,patents,copyrights


# Thanks Note

Thanks Prof Gallemore, for giving this assignment. I learnt a lot about web-scraping from this. I am happy that I am able to complete the task in approximately 13 hrs starting from sctrach on webscrapping and HTML knowledge.